In [ ]:
#Importando Librerías
import re
import json
import time
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

In [ ]:
#Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
#Ingresando a página para obtener información de las partes de Jojo's
page_url = f'https://jojowiki.com/Category:JoJo%27s_Bizarre_Adventure_Parts'
driver.get(page_url)

In [ ]:
#Encontrando partes
jojo_parts = driver.find_elements(By.XPATH,'//*[@id="mw-content-text"]/div[1]/div[1]/div/div[2]/div')

In [ ]:
jojo_data = {}
for part in jojo_parts:
    
    title = part.text
    title_url = title.replace(' ','_')
    url = f'https://jojowiki.com/{title_url}'
    jojo_data[title]={'url':url}

In [ ]:
jojo_data

In [ ]:
for key,value in jojo_data.items():
    driver.get(value['url'])
    print(key,'->',value['url'])
    info = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/p[3]')
    cln_info = (re.sub("\[.*?\]","",info.text))
    jojo_data[key]['info'] = cln_info
    
    
    resources = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/aside/div')
    for resource in resources:
        temp = resource.text.split('\n',1)
        jojo_data[key][temp[0].lower()] = temp[-1].replace('\n',' ')
        print('\t',temp[0].lower(),'->',temp[-1].replace('\n',' '))
        
    characters = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/div[14]/div/div[2]/div')
    characters_href = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/div[14]/div/div[2]/div/div[3]/a')
    
    temp_dict = {}
    for info, url in zip(characters, characters_href):
        
        time.sleep(2)
        role_name = info.text.split('\n')
        role = role_name[0]
        name = role_name[1]
        url = url.get_attribute('href')
        
#         driver.get(url)
#         time.sleep(2)
        
#         resume_xpath = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/p[3]')
#         resume = resume_xpath.text
        temp_dict[name] = {'url':url, 'role':role}
        
    jojo_data[key]['characters'] = temp_dict
    # print(jojo_data[key])

In [ ]:
resume_xpath = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/p[3]')
resume = resume_xpath.text
resume

In [ ]:
pprint.pprint(jojo_data)

In [ ]:
df = pd.DataFrame(jojo_data).T.reset_index()
df.rename(columns={'index':'part'}, inplace=True)
df

In [ ]:
characters = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/div[14]/div/div[2]/div')
characters[1].text

In [ ]:
driver.get('https://jojowiki.com/Stardust_Crusaders')

In [ ]:
characters = driver.find_elements(By.CLASS_NAME, 'diamond2')
characters_href = driver.find_elements(By.CLASS_NAME, 'diamond2')

In [ ]:
data_list = characters[0].text.split('\n')
[data_list[i:i+3] for i in range(0, len(data_list), 3)]

In [ ]:
with open('jojo_data.json', 'w',encoding='utf-8') as file:
    json.dump(jojo_data, file, indent=4, sort_keys=False)